_Dependencies and library requirements_

_Loading in station data by year. Hierarchal dataframe with levels based on year_

_Due to station distance matrices being different sizes current load pattern is by manual selection of year._

In [62]:
year = 2017

start = NR_ridership.loc[year,:]['Start_id'].values
end = NR_ridership.loc[year,:]['End_id'].values

# Build a data frame with start terminal as indices, end terminal as columns filled with zeros
station_ids = nr_station.loc[year,:].Terminal.values
travel_df = pd.DataFrame(columns = station_ids, index = station_ids).fillna(0)

for x in range(len(start)) :
    travel_df.loc[start[x], end[x]] += 1

In [63]:
clus_array = nr_station.Cluster.values
term_cluster_pair = {}
for x in range(len(station_ids)) :
    term_cluster_pair[station_ids[x]] = clus_array[x]

In [64]:
'''
Build this better using lambda mapping and dictionaries
'''
travel_to = pd.DataFrame(columns=['to_0', 'to_1', 'to_2', 'to_3', 'to_4', 'to_5'], \
                              index=station_ids).fillna(0)

for x in station_ids :
    if term_cluster_pair[x] == 0:
        travel_to['to_0'] += (travel_df[x].values)
    elif term_cluster_pair[x] == 1:
        travel_to['to_1'] += (travel_df[x].values)
    elif term_cluster_pair[x] == 2:
        travel_to['to_2'] += (travel_df[x].values)
    elif term_cluster_pair[x] == 3:
        travel_to['to_3'] += (travel_df[x].values)
    elif term_cluster_pair[x] == 4:
        travel_to['to_4'] += (travel_df[x].values)
    elif term_cluster_pair[x] == 5:
        travel_to['to_5'] += (travel_df[x].values)

In [65]:
travel_matrix = travel_df.values
from_matrix = travel_matrix.transpose(1,0)
from_df = pd.DataFrame(from_matrix, columns=station_ids, index=station_ids)

In [66]:
'''
Build this better using lambda mapping and dictionaries
'''

travel_from = pd.DataFrame(columns=['from_0', 'from_1', 'from_2', 'from_3', 'from_4', 'from_5'], \
                              index=station_ids).fillna(0)

for x in station_ids :
    if term_cluster_pair[x] == 0:
        travel_from['from_0'] += (from_df[x].values)
    elif term_cluster_pair[x] == 1:
        travel_from['from_1'] += (from_df[x].values)
    elif term_cluster_pair[x] == 2:
        travel_from['from_2'] += (from_df[x].values)
    elif term_cluster_pair[x] == 3:
        travel_from['from_3'] += (from_df[x].values)
    elif term_cluster_pair[x] == 4:
        travel_from['from_4'] += (from_df[x].values)
    elif term_cluster_pair[x] == 5:
        travel_from['from_5'] += (from_df[x].values)

In [67]:
to_summarize = pd.merge(travel_from, travel_to, left_index=True, right_index=True, how='outer').fillna(0)
to_summarize = pd.merge(to_summarize, nr_station.loc[year,:], left_index=True, right_on='Terminal')

In [68]:
intra_to_summary = pd.DataFrame(columns=['clus0', 'clus1', 'clus2','clus3', 'clus4', 'clus5', 'pct_intra'],\
                    index=['Cluster_'+str(x) for x in np.unique(to_summarize.Cluster.values)]).fillna(0)

for x in np.unique(to_summarize.Cluster.values) :
    intra_to_summary.loc['Cluster_'+str(x)]['clus0'] = to_summarize[to_summarize.Cluster == x]['to_0'].values.sum()
    intra_to_summary.loc['Cluster_'+str(x)]['clus1'] = to_summarize[to_summarize.Cluster == x]['to_1'].values.sum()
    intra_to_summary.loc['Cluster_'+str(x)]['clus2'] = to_summarize[to_summarize.Cluster == x]['to_2'].values.sum()
    intra_to_summary.loc['Cluster_'+str(x)]['clus3'] = to_summarize[to_summarize.Cluster == x]['to_3'].values.sum()
    intra_to_summary.loc['Cluster_'+str(x)]['clus4'] = to_summarize[to_summarize.Cluster == x]['to_4'].values.sum()
    intra_to_summary.loc['Cluster_'+str(x)]['clus5'] = to_summarize[to_summarize.Cluster == x]['to_5'].values.sum()
    intra_to_summary.loc['Cluster_'+str(x)]['pct_intra'] = (intra_to_summary.loc['Cluster_'+str(x)]['clus'+str(x)]\
                                                            / intra_to_summary.loc['Cluster_'+str(x),:].values.sum())\
                                                            *100
        
intra_from_summary = pd.DataFrame(columns=['clus0', 'clus1', 'clus2','clus3', 'clus4', 'clus5', 'pct_intra'],\
                    index=['Cluster_'+str(x) for x in np.unique(to_summarize.Cluster.values)]).fillna(0)

for x in np.unique(to_summarize.Cluster.values) :
    intra_from_summary.loc['Cluster_'+str(x)]['clus0'] = to_summarize[to_summarize.Cluster == x]['from_0'].values.sum()
    intra_from_summary.loc['Cluster_'+str(x)]['clus1'] = to_summarize[to_summarize.Cluster == x]['from_1'].values.sum()
    intra_from_summary.loc['Cluster_'+str(x)]['clus2'] = to_summarize[to_summarize.Cluster == x]['from_2'].values.sum()
    intra_from_summary.loc['Cluster_'+str(x)]['clus3'] = to_summarize[to_summarize.Cluster == x]['from_3'].values.sum()
    intra_from_summary.loc['Cluster_'+str(x)]['clus4'] = to_summarize[to_summarize.Cluster == x]['from_4'].values.sum()
    intra_from_summary.loc['Cluster_'+str(x)]['clus5'] = to_summarize[to_summarize.Cluster == x]['from_5'].values.sum()
    intra_from_summary.loc['Cluster_'+str(x)]['pct_intra'] = (intra_from_summary.loc['Cluster_'+str(x)]['clus'+str(x)]\
                                                              / intra_from_summary.loc['Cluster_'+str(x),:].values.sum())\
                                                              *100


In [69]:
print('To-From')
print(intra_to_summary,'\n')
print('From-To')
print(intra_from_summary)

To-From
           clus0  clus1  clus2  clus3  clus4  clus5  pct_intra
Cluster_0  63163  26517  25154  22719  15176  47181         31
Cluster_1  12393     95   2728   5269     35    141          0
Cluster_2   4342   1003   4009   1219    560   1123         32
Cluster_3  27202   2340   6996  10078   2988   7519         17
Cluster_4  30507  58893   4104  14644  24707  18170         16
Cluster_5   3329   4757   3699   2661   2215   2267         11 

From-To
           clus0  clus1  clus2  clus3  clus4  clus5  pct_intra
Cluster_0  62361  27216  23759  21372  15767  45761         31
Cluster_1  12105     79   3067   5409     32     72          0
Cluster_2   4112   1041   4131   1346    502   1154         33
Cluster_3  27773   2437   7535  10220   3530   8693         16
Cluster_4  30432  58319   3537  15540  24847  17804         16
Cluster_5   3487   4797   3744   2833   2301   2788         13


In [70]:
test1 = intra_to_summary.drop(['clus0', 'clus3', 'clus4'], axis=1).rename(\
                                                    {"clus1":'stp', 'clus2':'midway','clus5':'cedar'}, axis=1)
test2 = intra_from_summary.drop(['clus0', 'clus3', 'clus4'], axis=1).rename(\
                                                    {"clus1":'stp', 'clus2':'midway','clus5':'cedar'}, axis=1)

In [71]:
test1 = test1.drop(['Cluster_0', 'Cluster_3', 'Cluster_4'], axis=0).rename(\
                                                    {"Cluster_1":'stp', 'Cluster_2':'midway','Cluster_5':'cedar'}, axis=0)
test2 = test2.drop(['Cluster_0', 'Cluster_3', 'Cluster_4'], axis=0).rename(\
                                                    {"Cluster_1":'stp', 'Cluster_2':'midway','Cluster_5':'cedar'}, axis=0)

In [72]:
print('To-From')
print(test1,'\n')
print('From-To')
print(test2)

To-From
         stp  midway  cedar  pct_intra
stp       95    2728    141          0
midway  1003    4009   1123         32
cedar   4757    3699   2267         11 

From-To
         stp  midway  cedar  pct_intra
stp       79    3067     72          0
midway  1041    4131   1154         33
cedar   4797    3744   2788         13
